In [1]:
import datetime
import time
import sys
sys.path.append(r'/home/hengtian/lucy/factor-analyzer-v4/')
import pymysql
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import multiprocessing
import os
import queue
import tqdm
# from src import data 
# from src import analyzer
sys.path.append(os.path.abspath('/home/hengtian/kate/'))
import get_factor_tools as gft
# import datetime as dt

import plotly.graph_objects as go
from plotly.subplots import make_subplots

# import cufflinks as cf
# cf.go_offline()

sys.path.append(r'/home/hengtian/lucy/PortfolioOptimizer/')
from PortfolioOptimizerMaster.src import tools

from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression

In [ ]:
#近N个交易日内出现技术指标信号（N=60）
#技术指标信号日到今日的涨跌幅
# 一致预期、投资评级变动、未来增速、长研报因子值
# 当前单季成长、前一期单季成长因子值
#护城河因子值

In [148]:
def multi_process(stock_list, func,pro_num):
    q = multiprocessing.Manager().Queue()
    param = []
    for stock in stock_list:
        param.append((stock,q))
    # print(param)
    #task
    with multiprocessing.Pool(processes =pro_num) as pool:
        pool.starmap(func,param)

    #extract
    elements = []
    while q.qsize():
        elements.append(q.get())
    return elements

# 基本面因子

In [2]:
"""
df = 所有信号日的技术指标
oval = 每只股票每天的技术指标
"""
#每个日期往后找60个交易日
df=pd.read_csv('/home/hengtian/kate/signal/quant_signal/feature_selected_v7.csv',index_col =0)
oval = pd.read_csv('/home/hengtian/kate/signal/feature_selection/feature_selection_v4.csv',index_col =0)

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/numpy/lib/arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [5]:
ovlp = list(set(df['stock_code']))

In [3]:
#一致预期变动y1
npc_y1 = pd.read_csv('/home/hengtian/kate/signal/data/npc_y1.csv',index_col=0)
#一致预期变动y2
npc_y2 = pd.read_csv('/home/hengtian/kate/signal/data/npc_y2.csv',index_col=0)
#投资评级变动
rate = pd.read_csv('/home/hengtian/kate/signal/data/rate.csv',index_col=0)
#护城河因子
moat = pd.read_csv('/home/hengtian/kate/signal/data/moat.csv',index_col=0)
#未来增速
np_g1 = pd.read_csv('/home/hengtian/kate/signal/data/np_g1.csv',index_col=0)

In [4]:
lr = pd.read_csv('/home/hengtian/kate/signal/data/lr.csv',index_col=0)
#当前单季成长
growth_cur = pd.read_csv('/home/hengtian/kate/signal/data/growth_cur.csv',index_col=0)
#前一期单季成长因子值
growth_prev = pd.read_csv('/home/hengtian/kate/signal/data/growth_prev.csv',index_col=0)

In [6]:
lr = lr.loc['2013-01-01':]
extra = list(set(ovlp).difference(set(lr.columns)))
df1 = pd.DataFrame(np.empty((len(lr),len(extra)),dtype=object),columns = extra,index = lr.index)
lr = pd.concat([lr,df1],axis =1)

In [7]:
npc_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/npc_rank.csv',index_col=0)

In [8]:
npcy2_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/npcy2_rank.csv',index_col=0)

In [9]:
np_g1_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/np_g1_rank.csv',index_col=0)

In [10]:
moat_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/moat_rank.csv',index_col=0)

In [11]:
rate_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/rate_rank.csv',index_col=0)

In [12]:
lr_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/lr_rank.csv',index_col=0)

In [13]:
lr_rank = lr_rank.loc['2013-01-01':]
extra = list(set(ovlp).difference(set(lr_rank.columns)))
df1 = pd.DataFrame(np.empty((len(lr_rank),len(extra)),dtype=object),columns = extra,index = lr_rank.index)
lr_rank = pd.concat([lr_rank,df1],axis =1)

In [14]:
growth_prev_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/growth_prev_rank.csv',index_col=0)

In [15]:
growth_cur_rank = pd.read_csv('/home/hengtian/kate/signal/data/basic_rank/growth_cur_rank.csv',index_col=0)

In [16]:
npc_y1 = npc_y1[ovlp]
npc_y2 =npc_y2[ovlp]
rate = rate[ovlp]
moat = moat[ovlp]
np_g1=np_g1[ovlp]

In [17]:
growth_cur =growth_cur[ovlp]
growth_prev = growth_prev[ovlp]

In [18]:
npc_rank = npc_rank[ovlp]
npcy2_rank = npcy2_rank[ovlp]
np_g1_rank=np_g1_rank[ovlp]
moat_rank=moat_rank[ovlp]
rate_rank=rate_rank[ovlp]
lr_rank=lr_rank[ovlp]
growth_prev_rank=growth_prev_rank[ovlp]
growth_cur_rank=growth_cur_rank[ovlp]

In [19]:
lr = lr[ovlp]

In [20]:
lr.replace(to_replace=[None], value=np.nan, inplace=True)

In [21]:
df=df.rename(columns={'stock_code':'stock','date':'dates'})
oval = oval.rename(columns={'stock_code':'stock','date':'dates'})

# 找到信号日后90天

In [22]:
def find_days(stock,q):
    temp=df[df.stock==stock]
    atemp = oval[oval.stock==stock]
    all_days=[]
    for date in temp.dates:
        pos = list(atemp.dates.values).index(date)
        try:
            d=list(atemp.dates.values)[pos:pos+90]
        except IndexError:
            d = list(atemp.dates.values)[pos:]
        all_days+=d
    all_days = list(set(all_days))
    return q.put({stock:all_days})

In [23]:
elements = multi_process(set(df.stock),find_days,20)

In [24]:
final ={}
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    dates = elements[i][stock]
    final[stock]=dates

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [25]:
comp_dict= {
    'npc_y1':npc_y1,
    'npc_rank':npc_rank,
    'npc_y2':npc_y2,
    'npcy2_rank':npcy2_rank,
    'rate':rate,
    'rate_rank':rate_rank,
    'moat':moat,
    'moat_rank':moat_rank,
    'np_g1':np_g1,
    'np_g1_rank':np_g1_rank,
    'lr':lr,
    'lr_rank':lr_rank,
    'growth_cur':growth_cur,
    'growth_cur_rank':growth_cur_rank,
    'growth_prev':growth_prev,
    'growth_prev_rank':growth_prev_rank
}

In [26]:
def find_factors(stock,q):
    dates =sorted(final.get(stock))
    factor = []
    col =[]
    for comp_name,comp_df in comp_dict.items():
        data = comp_df[stock]
        val = data.loc[dates]
        factor.append(val)
        col.append(comp_name)
    temp = pd.concat(factor,axis = 1)
    temp.columns = col
    return q.put({stock:temp})
#     return(temp)

In [27]:
elements = multi_process(final.keys(), find_factors,20)

In [28]:
final =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock].reset_index().rename(columns={'index' : 'dates'})
    exp['stock'] = [stock]*len(exp)
    final.append(exp)

In [29]:
f_df = pd.concat(final)

In [ ]:
f_df.to_csv('basic_features_v3.csv')

# 100天前700日max & 100日min

In [ ]:
# cop = df.reset_index()
#最低价
lowest = pd.read_csv('/home/hengtian/kate/signal/data/quant_data/lowest_post.csv',index_col=0)
stocks = set(df.stock)
lowest = lowest[stocks]

In [ ]:
#最高价
highest = pd.read_csv('/home/hengtian/kate/signal/data/quant_data/highest_post.csv',index_col=0)
stocks = set(df.stock)
highest =highest[stocks]

In [ ]:
cop =df.set_index(['stock','dates'])

In [ ]:
mx7 = highest.rolling(700,min_periods =300).max().shift(100).unstack().rename('max_700')
sel_max = mx7[mx7.index.isin(cop.index)]

In [ ]:
sel_max = sel_max.reset_index()
# sel_max[sel_max.stock == '600410.XSHG']

In [ ]:
sel_max.columns = ['stock','dates','max_700']

# 100日min，第二天开盘，当天收盘

In [ ]:
min1 = lowest.rolling(100,min_periods =50).min().unstack().rename('min_100')
sel_min = min1[min1.index.isin(cop.index)]

In [ ]:
sel_min = sel_min.reset_index()
sel_min.columns = ['stock','dates','min_100']

In [ ]:
close_price = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv",index_col = 0)
close_prie= close_price.replace(0,np.nan)

close_price = close_price[:'2021-07-13']
close_price = close_price[stocks]
close = close_price.unstack()
close_price = close[close.index.isin(cop.index)]

In [ ]:
close_price = close_price.reset_index()
close_price.columns=['stock','dates','close_price']

In [ ]:
sig_dict = {
    'max_700':sel_max,
    'min_100':sel_min,
    'close_price':close_price,
#     'days_btw_max':final_m,
}

In [ ]:
for sig_name,sig_df in sig_dict.items():
#     sig_df = sig_df.reset_index()
#     sig_df.columns=['stock','dates',sig_name]
    df = df.merge(sig_df,on =['stock','dates'])
#     df[sig_name].fillna(method='ffill',inplace=True)

In [ ]:
data_l = data[data.dates >'2016']

In [ ]:
data.to_csv('overall_signals_updated.csv')
data_l.to_csv('overall_signals_2016_up.csv')

In [ ]:
f_cop = pd.merge(f_cop,sel_price,how = 'left',on =['stock','dates'])
f_cop['close_price'].fillna(method='ffill',inplace=True)

In [ ]:
op_price = pd.read_csv("/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv",index_col = 0)
op_prie= op_price.replace(0,np.nan)

op_price = op_price['2013':'2021-07-18'].shift(-1)
op_price = op_price[stocks]
op = op_price.unstack()
op_price = op[op.index.isin(cop.index)]

In [ ]:
op_price = op_price.reset_index()
op_price.columns = ['stock','dates','buy_price']

In [ ]:
df = df.merge(op_price, how ='left',on = ['stock','dates'])

# 信号日距离100天前的700日max的天数

In [ ]:
# max_date =pd.Series()
def btw_max(stock,q):
    temp_h = highest[stock]
#     print(temp_h)
    temp_m = sel_max[sel_max.stock==stock]
    max_days = []
    for date in temp_m.dates:
        temp = temp_h[:date].tail(800)
        begin = np.argmax(temp[:-100])
        itval =len(temp[begin:])-1
        max_days.append([date,itval])
#     m_df =pd.DataFrame(max_days)
    return q.put({stock:max_days})


In [ ]:
elements = multi_process(set(df.stock),btw_max,20)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = pd.DataFrame(elements[i][stock])
    exp['stock']=stock
#     print(t_df)
    elmts.append(exp)

In [ ]:
final_m = pd.concat(elmts)

In [ ]:
final_m.columns=['dates','days_btw_max','stock']
# final_m[final_m.days_btw_max<100]

In [ ]:
df = df.merge(final_m,on=['stock','dates'])

# 未来收益率

In [178]:
price = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/open_price_post.csv',index_col =0)
price = price['2013':'2021-07-18']

In [180]:
price = price[ovlp]

In [181]:
#基准收益
zz_500 = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/bmk_open.csv',index_col =0)
zz_500= zz_500['2013':'2021-07-18']
zz_500=zz_500['000905.XSHG']
zz_rtn =(zz_500/zz_500.shift(1)-1).shift(-2)

In [126]:
"""
计算每只股票的未来1天收益
"""
def ft_rtn(stock,q):
    dates = np.array(dfv5[dfv5['stock']==stock]['dates'].values)
#     dates = [i if (i in dates)& (i<'2021-07-12')]
    op = price[stock]
    rtn_f = calc_rtn(1,dates,op)
#     rtn_f.columns ='rtn_1'
    rtn_f['dates']=dates
    rtn_f['stock'] = stock
    return q.put({stock:rtn_f})
#     tot.append(rtn_f)

In [184]:
"""
计算n天之后的收益
"""
def calc_rtn(days,dates,op):
    tmr =[list(price.index).index(i)+1 for i in dates]
#     tmr = [i if (i in tmr)&(i<2071) for i in tmr]
    ftr = [list(price.index).index(i)+1+days for i in dates]
    ftr = [i if (i in ftr)&(i<2075) else 2074 for i in ftr]
    return pd.DataFrame(op.iloc[ftr].values/op.iloc[tmr].values-1)

In [185]:
elements = multi_process(set(dfv5['stock']),ft_rtn,20)

In [186]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    elmts.append(exp)

In [187]:
rtns = pd.concat(elmts)

# dff.loc['000026.XSHE']

In [188]:
rtns.columns = ['rtn_1','dates','stock']

In [192]:
"""
计算每只股票未来一天的超额收益
"""
def alpha_rtn(stock,q):
    rtn_days =dfv8[dfv8.stock==stock]['dates'].values
    zz_rt = zz_rtn[rtn_days].values
    rtn = dfv8[dfv8.stock==stock]['rtn_1'].values - zz_rt
    alpha = pd.DataFrame([rtn_days,rtn]).T
#     print(alpha)
    return q.put({stock:alpha})

In [193]:
elements = multi_process(set(dfv8['stock']),alpha_rtn,30)

In [194]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    exp['stock'] = stock
    elmts.append(exp)

In [195]:
rtns = pd.concat(elmts)
rtns.columns =['dates','alpha_1','stock']
rtns

,dates,alpha_1,stock
0,2018-05-22,0.0293619,002816.XSHE
1,2018-05-23,0.0019058,002816.XSHE
2,2018-05-24,0.0117583,002816.XSHE
3,2018-05-25,-0.00856598,002816.XSHE
4,2018-05-28,-0.01971,002816.XSHE
...,...,...,...
258,2021-03-31,0.00090082,002003.XSHE
259,2021-04-01,-0.0486149,002003.XSHE
260,2021-04-02,-0.00367348,002003.XSHE
261,2021-04-06,0.00869864,002003.XSHE


In [197]:
"""
所有满足利空事件统计指标的股票和日期
"""
dfv6 = dfv9[(dfv9['npc_y1'].isnull()==False)&(dfv9['npc_y2'].isnull()==False)&(dfv9['np_g1'].isnull()==False)&(dfv9['growth_cur']>0.1)]
dfv7 =dfv6[(dfv6['is_limit']==0)&(dfv6['is_st']==0)]

In [198]:
df10 = dfv7[(dfv7['buy/sig_close']<-0.02)&(dfv7['interval_days']>1)&(dfv7['interval_days']<60)]

In [200]:
df10.to_csv('signal_selected_0809.csv')

# 距离信号日天数

In [ ]:
data = pd.read_csv('/home/hengtian/kate/signal/overall_signals_updated.csv')

In [51]:
ftr_days =dfv3[['stock','dates']]
all_days =oval[['stock','dates']].set_index('dates')
sig_days= df[['stock','dates']]

In [53]:
"""
找到每天所对应的信号日

"""
def find_sigday(stock,q):
    temp_f =ftr_days[ftr_days.stock==stock]
    temp_s = sig_days[sig_days.stock==stock]
    sig = []
    for date in temp_f.dates:
        sig_day =temp_s[temp_s.dates<=date].iloc[-1].values[1]
        sig.append([date,sig_day])
    sig = pd.DataFrame(sig)
    return q.put({stock:sig})

In [ ]:
elements = multi_process(set(sig_days.stock),find_sigday,20)

In [59]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    exp['stock']=stock
#     print(t_df)
    elmts.append(exp)

In [61]:
ftr = pd.concat(elmts)
ftr.columns =['dates','sig_date','stock']

In [ ]:
df6.set_index(['stock','dates'],inplace =True)
df.set_index(['stock','dates'],inplace =True)
sig_days =df6[df6.index.isin(df.index)].reset_index()

In [ ]:
all_days.set_index('dates',inplace = True)

In [201]:
"""
计算每天与信号日之间间隔的交易天数（不包含其中一天）

"""

def itval_days(stock,q):
    temp_re=[]
    temp_a =all_days[all_days.stock ==stock]
    temp_f =ftr[ftr.stock==stock].drop('stock',axis = 1)
    for i in range(len(temp_f)):
        end =temp_f.iloc[i,:].values[0]
        begin= temp_f.iloc[i,:].values[1]
        temp_re.append(len(temp_a.loc[begin:end])-1)
    return q.put({stock:temp_re})


In [ ]:
elements = multi_process(set(ftr.stock),itval_days,20)

In [65]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    t_df =pd.DataFrame(exp)
#     print(t_df)
    t_df['stock']=stock
#     print(t_df)
    elmts.append(t_df)

In [66]:
fi=pd.concat(elmts)
fi

,0,stock
0,0,603713.XSHG
1,1,603713.XSHG
2,2,603713.XSHG
3,0,603713.XSHG
4,0,603713.XSHG
...,...,...
29,29,002961.XSHE
30,30,002961.XSHE
31,31,002961.XSHE
32,32,002961.XSHE


In [67]:
# ftr_days.set_index('stock',inplace=True)
fi.set_index('stock',inplace=True)

In [69]:
"""
将找到的天数与原本的dataframe结合
"""
def concat(stock,q):
    temp_f =dfv3[dfv3.index==stock]
    temp_i = fi[fi.index==stock]
    temp =pd.concat([temp_f,temp_i],axis =1)
    q.put({stock:temp})
# final_df =pd.concat(final_interval)
# final_df

In [ ]:
elements = multi_process(set(dfv3.index),concat,20)

In [71]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = elements[i][stock]
    exp['stock']=stock
#     print(t_df)
    elmts.append(exp)

In [72]:
final_df = pd.concat(elmts)

In [79]:
final_df.rename(columns={0:'interval_days'},inplace=True)
# final_df.drop('stock',axis =1,inplace=True)

# 中证500 特征

In [ ]:
zz_500 =pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/is_zz800.csv',index_col =0)

In [ ]:
zz_highest =gft.get_index_factor('highest_index',start_date =datetime.date(2013, 1, 1),end_date = datetime.date(2021, 7, 13))
zz_highest.to_csv('highest_index.csv')

In [ ]:
zz_lowest =gft.get_index_factor('lowest_index',start_date =datetime.date(2013, 1, 1),end_date = datetime.date(2021, 7, 13))
zz_lowest.to_csv('lowest_index.csv')

In [ ]:
zz_highest =zz_highest['000905.XSHG']
zz_lowest= zz_lowest['000905.XSHG']

In [ ]:
zz_lowest.index = zz_lowest.index.to_series().apply(lambda x :x.strftime("%Y-%m-%d"))

In [ ]:
zz_highest.index = zz_highest.index.to_series().apply(lambda x :x.strftime("%Y-%m-%d"))

In [ ]:
# 中证特征思路
"""
每个stock的每个信号日定位回low_post中的前20天（不包括信号日当天）
找到20天中的min 的日期，去找中证500的价格（中证500当天的lowest）
当天的max/中证的价格

"""
def zz_rtn(stock,q):
    temp =df[df.stock==stock]
    temp_l = lowest[stock]
    rtns = []
    for date in temp.dates:
#         print(date)
        idx =list(temp_l.index).index(date)
        min_idx =np.argmin(temp_l.iloc[idx-20:idx])
        min_date = temp_l.iloc[idx-20:idx].index[min_idx]
        zz_min = zz_lowest[min_date]
#         print(zz_min)
        zz_high =zz_highest[date]
        rtn = zz_high/zz_min
#         print(rtn)
        rtns.append([date,rtn])
#         break
#     return rtns
    return q.put({stock:rtns})

In [ ]:
elements = multi_process(set(df.stock),zz_rtn,20)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = pd.DataFrame(elements[i][stock])
    exp['stock']=stock
#     print(t_df)
    elmts.append(exp)

In [ ]:
zz_min20 = pd.concat(elmts)
zz_min20.columns = ['dates','zz_max_vs_min_20','stock']

In [ ]:
df = df.merge(zz_min20,on = ['dates','stock'])

In [ ]:
"""
每个stock的每个信号日100天前的700天中最高价的日期（不包括信号日当天）
去找当天中证500的价格（中证500当天的max）；
每个stock 的每个信号日100天中的最低价的日期，去找当天中证500的价格（中证500当天的min）
中证的max/min

"""
def zz_max700(stock,q):
    temp_h = highest[stock]
#     print(temp_h)
    temp_m = df[df.stock==stock]
    ratio = []
    for date in temp_m.dates:
        temp = temp_h[:date].tail(800)
        idx = np.argmax(temp.iloc[:-100])
        max_date = temp.iloc[:-100].index[idx]
        min_idx = np.argmin(temp.tail(100))
        min_date = temp.tail(100).index[min_idx]
        zz_ratio = zz_highest[max_date]/zz_lowest[min_date]
        ratio.append([date,zz_ratio])
#     m_df =pd.DataFrame(max_days)
    return q.put({stock:ratio})


In [ ]:
elements = multi_process(set(df.stock),zz_max700,20)

In [ ]:
elmts =[]
for i in range(len(elements)):
    stock = list(elements[i].keys())[0]
    exp = pd.DataFrame(elements[i][stock])
    exp['stock']=stock
#     print(t_df)
    elmts.append(exp)

In [ ]:
zz_max = pd.concat(elmts)

In [ ]:
zz_max.columns = ['dates','zz_max700_min100','stock']

In [ ]:
df = df.merge(zz_max,on=['stock','dates'])

# 涨停

In [31]:
#当日涨停价
high_l = pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/high_limit.csv',index_col =0)

In [32]:
#当日收盘价（未复权）
close_non=pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/close_price.csv',index_col =0)

In [33]:
"""
结合涨停价和收盘价，相等的为涨停股票&日期
"""
close_non = close_non.unstack().rename('close').reset_index()
close_non.columns = ['stock','dates','close_price']
high_l = high_l.unstack().rename('high_limit').reset_index()
high_l.columns = ['stock','dates','high_limit']
combo = pd.merge(close_non, high_l, on = ['stock','dates'])
limitup = combo[combo.close_price==combo.high_limit]

# 第二天涨停

In [ ]:
high_l_shift =pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/high_limit.csv',index_col =0)
high_l_shift = high_l_shift.shift(-1)
high_l_shift = high_l_shift.unstack().reset_index()
high_l_shift.columns = ['stock','dates','high_limit_shifted']
signals = signals.merge(high_l_shift,how = 'left',on = ['stock','dates'])

In [ ]:
signals.drop(['close_price','high_limit'],axis =1,inplace = True)

In [ ]:
limitup.reset_index(inplace =True)

In [37]:
limitup.set_index(['stock','dates'],inplace = True)

In [85]:
close_price_post =pd.read_csv('/home/hengtian/lucy/factor_service/basic_data/close_price_post.csv',index_col =0)
close_price_p = close_price_post.unstack().reset_index()
close_price_p.columns = ['stock','dates','close_price_post']
sig_close =df.merge(close_price_p,how='left',on = ['stock','dates'])

In [38]:
limit_list = df[df.index.isin(limitup.index)]
limit_list['is_limit'] = 1
temp_limit = limit_list['is_limit']
temp_limit=temp_limit.reset_index()
# temp_limit =temp_limit[temp_limit.dates>'2016']
# temp_limit

/home/hengtian/anaconda3/envs/lucy/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [46]:
dfv2 = f_df.merge(dfv1[['stock','dates','is_limit']],how= 'left',on=['stock','dates'])

In [49]:
dfv2['is_limit'].fillna(method='ffill',inplace = True)
# df4 =df3[df3.is_limit==0]